In [11]:
import numpy as np
import cv2
import face_recognition
import os
from datetime import datetime

path="data"
images=[]
ClassNames=[]

mylist=os.listdir(path)
for cls in mylist:
    currentImg=cv2.imread(f"{path}/{cls}")
    images.append(currentImg)
    ClassNames.append(os.path.splitext(cls)[0])
# print(ClassNames)
# cv2.imshow("image",images[2])
# cv2.waitKey(0)

def encoder(images):
    encode_list=[]
    for img in images:
        img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode= face_recognition.face_encodings(img)[0]
        encode_list.append(encode)
    return encode_list

def mark_attendence(name):
    with open("attendence.csv", 'r+') as f:
        myDataList=f.readlines()
        nameList=[]
        for line in myDataList:
            entry = line.split(',')
            nameList.append(entry[0])
        if name not in nameList:
            now= datetime.now()
            datestring=now.strftime('%H:%M:%S')
            f.writelines(f"\n{name},{datestring}")
            
#         print(myDataList)

    


encodeList= encoder(images)
# print(len(encodeList))
print("Encoding Complete")



cap=cv2.VideoCapture(0)

while True:
    success, img = cap.read()
    imgS= cv2.resize(img,(0,0),None,0.25,0.25)
    imgS=cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)
    
    face_Current_Frame= face_recognition.face_locations(imgS)
    encoding_Current_Frame= face_recognition.face_encodings(imgS,face_Current_Frame)
    
    for encode_face, face_loc in zip(encoding_Current_Frame,face_Current_Frame):
        matches=face_recognition.compare_faces(encodeList,encode_face)
        face_Distance=face_recognition.face_distance(encodeList,encode_face)
        match_index=np.argmin(face_Distance)
        if matches[match_index]:
            name=ClassNames[match_index].upper() 
            mark_attendence(name)
            print(f"Attandence Marked for {name}")
            y1,x2,y2,x1=face_loc
            y1,x2,y2,x1=y1*4,x2*4,y2*4,x1*4
            cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,0),2)
            cv2.rectangle(img,(x1,y2-35),(x2,y2),(0,255,0),cv2.FILLED)
            cv2.putText(img, name,(x1+6,y2-6),cv2.FONT_HERSHEY_COMPLEX,0.8,(255,255,255),2)
            
            
    cv2.imshow("video",img)
    if cv2.waitKey(1) & 0xff ==ord('q'):
        break

Encoding Complete
Attandence Marked for NADEEM
Attandence Marked for NADEEM
Attandence Marked for NADEEM
Attandence Marked for NADEEM
Attandence Marked for NADEEM
Attandence Marked for NADEEM
Attandence Marked for NADEEM
Attandence Marked for NADEEM
Attandence Marked for NADEEM
Attandence Marked for NADEEM
Attandence Marked for NADEEM
Attandence Marked for NADEEM
Attandence Marked for NADEEM
Attandence Marked for NADEEM
Attandence Marked for NADEEM
Attandence Marked for NADEEM
Attandence Marked for NADEEM
Attandence Marked for NADEEM
Attandence Marked for NADEEM
Attandence Marked for NADEEM
Attandence Marked for NADEEM
Attandence Marked for NADEEM
Attandence Marked for NADEEM
Attandence Marked for NADEEM
Attandence Marked for NADEEM
Attandence Marked for NADEEM
Attandence Marked for NADEEM
Attandence Marked for NADEEM
Attandence Marked for NADEEM
Attandence Marked for NADEEM
Attandence Marked for NADEEM
Attandence Marked for NADEEM
Attandence Marked for NADEEM
Attandence Marked for NAD

In [ ]:
!pip install CMake==3.22.2

In [ ]:
!cmake --version

In [ ]:
!pip uninstall dlib

In [ ]:
!pip install wheel setuptools


In [ ]:
!pip install mediapipe --user


In [1]:
import cv2
import mediapipe as mp
import os
from datetime import datetime
import numpy as np

# Initialize Mediapipe Face Recognition Tools
mp_face_detection = mp.solutions.face_detection
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils

path = "data"
images = []
class_names = []

# Load Images and Extract Class Names
my_list = os.listdir(path)
for cls in my_list:
    current_img = cv2.imread(f"{path}/{cls}")
    images.append(current_img)
    class_names.append(os.path.splitext(cls)[0])

# Encode Images (Using Simple Flattening as an Alternative)
def encoder(images):
    encode_list = []
    for img in images:
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        flattened_img = img_rgb.flatten() / 255.0  # Flatten and normalize
        encode_list.append(flattened_img)
    return np.array(encode_list)

# Mark Attendance
def mark_attendance(name):
    with open("attendance.csv", 'r+') as f:
        data_list = f.readlines()
        name_list = [line.split(',')[0] for line in data_list]
        
        if name not in name_list:
            now = datetime.now()
            date_string = now.strftime('%H:%M:%S')
            f.writelines(f"\n{name},{date_string}")

# Encode the images
encode_list_known = encoder(images)
print("Encoding Complete")

# Start Video Capture
cap = cv2.VideoCapture(0)

while True:
    success, img = cap.read()
    if not success:
        break
    
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Use Mediapipe for Face Detection
    with mp_face_detection.FaceDetection(min_detection_confidence=0.5) as face_detection:
        results = face_detection.process(img_rgb)
        
        if results.detections:
            for detection in results.detections:
                bboxC = detection.location_data.relative_bounding_box
                h, w, _ = img.shape
                x1, y1 = int(bboxC.xmin * w), int(bboxC.ymin * h)
                x2, y2 = int((bboxC.xmin + bboxC.width) * w), int((bboxC.ymin + bboxC.height) * h)

                # Extract face region and encode it
                face_region = img_rgb[y1:y2, x1:x2].flatten() / 255.0
                face_region = np.array([face_region])  # Reshape for comparison

                # Calculate Euclidean Distance to Known Encodings
                distances = np.linalg.norm(encode_list_known - face_region, axis=1)
                match_index = np.argmin(distances)

                if distances[match_index] < 0.6:  # Threshold for a match
                    name = class_names[match_index].upper()
                    mark_attendance(name)
                    print(f"Attendance Marked for {name}")

                    # Draw Bounding Box and Name
                    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(img, name, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

    # Display Video Feed
    cv2.imshow("Video", img)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (6,) + inhomogeneous part.

In [ ]:
!pip show mediapipe


In [4]:
import cv2
import mediapipe as mp
import numpy as np
import os
from datetime import datetime

# Initialize MediaPipe Face Detection and Drawing Utilities
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

# Path to known images
path = "data"
images = []
classNames = []

# Load and prepare known images
mylist = os.listdir(path)
for cls in mylist:
    currentImg = cv2.imread(f"{path}/{cls}")
    images.append(currentImg)
    classNames.append(os.path.splitext(cls)[0])

def encode_images(images):
    """Extract and store encodings using MediaPipe."""
    encodings = []
    with mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.5) as face_detection:
        for img in images:
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            results = face_detection.process(img_rgb)
            if results.detections:
                encodings.append(results.detections[0].location_data)
            else:
                print("No face detected. Skipping image...")
    return encodings

def mark_attendance(name):
    """Mark attendance in CSV."""
    with open("attendence.csv", 'r+') as f:
        myDataList = f.readlines()
        nameList = [line.split(',')[0] for line in myDataList]

        if name not in nameList:
            now = datetime.now()
            timestamp = now.strftime('%H:%M:%S')
            f.writelines(f"\n{name},{timestamp}")

# Encode known images
encoded_known_faces = encode_images(images)
print("Encoding Complete")

# Start webcam for real-time face detection
cap = cv2.VideoCapture(0)

with mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.5) as face_detection:
    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            print("Skipping empty frame...")
            continue

        img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_detection.process(img_rgb)

        if results.detections:
            for detection in results.detections:
                bboxC = detection.location_data.relative_bounding_box
                h, w, _ = frame.shape
                x1, y1 = int(bboxC.xmin * w), int(bboxC.ymin * h)
                x2, y2 = int((bboxC.xmin + bboxC.width) * w), int((bboxC.ymin + bboxC.height) * h)

                # Draw bounding box and label
                mp_drawing.draw_detection(frame, detection)
                name = "UNKNOWN"  # Default as unknown; you can add matching logic here.
                mark_attendance(name)

                print(f"Attendance marked for {name}")

        cv2.imshow("MediaPipe Face Detection", frame)
        if cv2.waitKey(1) & 0xFF == oqrd('q'):
            break

cap.release()
cv2.destroyAllWindows()


Encoding Complete
Attendance marked for UNKNOWN
Attendance marked for UNKNOWN
Attendance marked for UNKNOWN
Attendance marked for UNKNOWN
Attendance marked for UNKNOWN
Attendance marked for UNKNOWN
Attendance marked for UNKNOWN
Attendance marked for UNKNOWN
Attendance marked for UNKNOWN
Attendance marked for UNKNOWN
Attendance marked for UNKNOWN
Attendance marked for UNKNOWN
Attendance marked for UNKNOWN
Attendance marked for UNKNOWN
Attendance marked for UNKNOWN
Attendance marked for UNKNOWN
Attendance marked for UNKNOWN
Attendance marked for UNKNOWN
Attendance marked for UNKNOWN
Attendance marked for UNKNOWN
Attendance marked for UNKNOWN
Attendance marked for UNKNOWN
Attendance marked for UNKNOWN
Attendance marked for UNKNOWN
Attendance marked for UNKNOWN
Attendance marked for UNKNOWN
Attendance marked for UNKNOWN
Attendance marked for UNKNOWN
Attendance marked for UNKNOWN
Attendance marked for UNKNOWN
Attendance marked for UNKNOWN
Attendance marked for UNKNOWN
Attendance marked for 

In [9]:
import cv2
import mediapipe as mp
import numpy as np
import os
from datetime import datetime

# Initialize MediaPipe Face Detection and Drawing Utilities
mp_face_detection = mp.solutions.face_detection
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils

# Path to known images
path = "data"
images = []
classNames = []

# Load and prepare known images
mylist = os.listdir(path)
for cls in mylist:
    currentImg = cv2.imread(f"{path}/{cls}")
    images.append(currentImg)
    classNames.append(os.path.splitext(cls)[0])

def get_face_embedding(image):
    """Extract face embeddings using MediaPipe Face Mesh."""
    with mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1) as face_mesh:
        img_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(img_rgb)
        
        if results.multi_face_landmarks:
            return np.array([[landmark.x, landmark.y, landmark.z] for landmark in results.multi_face_landmarks[0].landmark]).flatten()
    return None

def mark_attendance(name):
    """Mark attendance in CSV."""
    with open("attendence.csv", 'r+') as f:
        myDataList = f.readlines()
        nameList = [line.split(',')[0] for line in myDataList]

        if name not in nameList:
            now = datetime.now()
            timestamp = now.strftime('%H:%M:%S')
            f.writelines(f"\n{name},{timestamp}")

# Encode known images
known_face_embeddings = [get_face_embedding(img) for img in images if get_face_embedding(img) is not None]
print("Encoding Complete")

# Start webcam for real-time face detection
cap = cv2.VideoCapture(0)

with mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.5) as face_detection:
    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            print("Skipping empty frame...")
            continue

        img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_detection.process(img_rgb)

        if results.detections:
            for detection in results.detections:
                bboxC = detection.location_data.relative_bounding_box
                h, w, _ = frame.shape
                x1, y1 = int(bboxC.xmin * w), int(bboxC.ymin * h)
                x2, y2 = int((bboxC.xmin + bboxC.width) * w), int((bboxC.ymin + bboxC.height) * h)

                # Extract face region
                face_img = frame[y1:y2, x1:x2]
                face_embedding = get_face_embedding(face_img)

                # Check if face_embedding is None
                if face_embedding is not None:
                    # Compare with known embeddings
                    distances = [np.linalg.norm(known_face - face_embedding) for known_face in known_face_embeddings]
                    min_distance_index = np.argmin(distances)

                    # Set a threshold for matching
                    if distances[min_distance_index] < 0.6:  # Adjust threshold as needed
                        name = classNames[min_distance_index].upper()
                    else:
                        name = "UNKNOWN"
                else:
                    name = "UNKNOWN"

                # Draw bounding box and label
                mp_drawing.draw_detection(frame, detection)
                mark_attendance(name)
                print(f"Attendance marked for {name}")

        cv2.imshow("MediaPipe Face Detection", frame)q
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


Encoding Complete
Attendance marked for UNKNOWN
Attendance marked for UNKNOWN
Attendance marked for UNKNOWN
Attendance marked for UNKNOWN
Attendance marked for UNKNOWN
Attendance marked for UNKNOWN
Attendance marked for UNKNOWN
Attendance marked for UNKNOWN
Attendance marked for UNKNOWN
Attendance marked for UNKNOWN
Attendance marked for UNKNOWN
Attendance marked for UNKNOWN
Attendance marked for UNKNOWN
Attendance marked for UNKNOWN
Attendance marked for UNKNOWN
Attendance marked for UNKNOWN
Attendance marked for UNKNOWN
Attendance marked for UNKNOWN
Attendance marked for UNKNOWN
Attendance marked for UNKNOWN
Attendance marked for UNKNOWN
Attendance marked for UNKNOWN
Attendance marked for UNKNOWN
Attendance marked for UNKNOWN
Attendance marked for UNKNOWN
Attendance marked for UNKNOWN
Attendance marked for UNKNOWN
Attendance marked for UNKNOWN
Attendance marked for UNKNOWN
Attendance marked for UNKNOWN
Attendance marked for UNKNOWN
Attendance marked for UNKNOWN
Attendance marked for 

In [17]:
import numpy as np
import cv2
import face_recognition
import os
from datetime import datetime, timedelta
import csv

path = "data"
images = []
ClassNames = []

# Load images and class names
mylist = os.listdir(path)
for cls in mylist:
    currentImg = cv2.imread(f"{path}/{cls}")
    images.append(currentImg)
    ClassNames.append(os.path.splitext(cls)[0])

def encoder(images):
    encode_list = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encode_list.append(encode)
    return encode_list

def create_csv_if_not_exists():
    if not os.path.exists("attendance.csv"):
        with open("attendance.csv", 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(["Date", "Name", "Entry Time", "Exit Time", "Reentry", "Final Exit Time"])
        print("Created 'attendance.csv' with required columns.")

def mark_attendance(name):
    current_date = datetime.now().strftime('%Y-%m-%d')
    current_time = datetime.now().strftime('%H:%M:%S')
    current_datetime = datetime.now()

    with open("attendance.csv", 'r+', newline='') as f:
        myDataList = f.readlines()
        nameList = []
        recordExists = False
        reentry = False
        final_exit_time_updated = False

        for line in myDataList[1:]:  # Skip the header
            entry = line.strip().split(',')
            record_date = entry[0]
            record_name = entry[1]
            entry_time = entry[2]
            exit_time = entry[3]

            # Check if the record already exists for today
            if record_name == name and record_date == current_date:
                recordExists = True
                entry_time_obj = datetime.strptime(entry_time, '%H:%M:%S')

                if exit_time == "":
                    # Check if 10 minutes have passed since entry time before marking exit
                    if current_datetime - entry_time_obj >= timedelta(minutes=10):
                        # Mark the exit time
                        myDataList[myDataList.index(line)] = f"{current_date},{name},{entry_time},{current_time},False,\n"
                        print(f"Exit marked for {name} at {current_time}")
                    else:
                        print(f"Exit not marked for {name}, less than 10 minutes since entry.")
                elif entry[4] == "False":
                    # Mark reentry and exit time if not already done
                    myDataList[myDataList.index(line)] = f"{current_date},{name},{entry_time},{exit_time},True,{current_time}\n"
                    reentry = True
                elif entry[4] == "True" and entry[5] == "":
                    # If reentry is already True and final exit is not marked, update final exit time
                    myDataList[myDataList.index(line)] = f"{current_date},{name},{entry_time},{exit_time},True,{current_time}\n"
                    final_exit_time_updated = True
                break

        if not recordExists:
            # If no record for today, mark entry
            f.writelines(f"{current_date},{name},{current_time},,,False,\n")
            print(f"Entry recorded for {name} at {current_time}")

        # Update the CSV file with new data
        f.seek(0)
        f.writelines(myDataList)

        if reentry:
            print(f"Reentry marked for {name}.")
        if final_exit_time_updated:
            print(f"Final exit time updated for {name}.")

# Create CSV if it doesn't exist
create_csv_if_not_exists()

encodeList = encoder(images)
print("Encoding Complete")

cap = cv2.VideoCapture(0)

# Dictionary to store the last time a person was recognized
last_recognized_time = {}

while True:
    success, img = cap.read()
    imgS = cv2.resize(img, (0, 0), None, 0.25, 0.25)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)

    face_Current_Frame = face_recognition.face_locations(imgS)
    encoding_Current_Frame = face_recognition.face_encodings(imgS, face_Current_Frame)

    for encode_face, face_loc in zip(encoding_Current_Frame, face_Current_Frame):
        matches = face_recognition.compare_faces(encodeList, encode_face)
        face_Distance = face_recognition.face_distance(encodeList, encode_face)
        match_index = np.argmin(face_Distance)

        if matches[match_index]:
            name = ClassNames[match_index].upper()
            current_time = datetime.now()

            # Check if the person was recognized within the last 20 seconds
            if name not in last_recognized_time or current_time - last_recognized_time[name] > timedelta(seconds=20):
                # Update the last recognized time for this person
                last_recognized_time[name] = current_time

                # Mark attendance
                mark_attendance(name)
                print(f"Attendance marked for {name}")

                # Draw a rectangle around the face
                y1, x2, y2, x1 = face_loc
                y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)
                cv2.putText(img, name, (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 0.8, (255, 255, 255), 2)

    cv2.imshow("video", img)
    if cv2.waitKey(1) & 0xff == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Encoding Complete
Attendance Marked for NADEEM


In [18]:
import numpy as np
import cv2
import face_recognition
import os
import pandas as pd
from datetime import datetime

path = "data"
images = []
ClassNames = []

# Check if the attendance.csv file exists; if not, create it with the required columns
attendance_file = "attendance.csv"
if not os.path.exists(attendance_file):
    # Create a new DataFrame with the required columns
    df = pd.DataFrame(columns=["Date", "Name", "Entry_Time", "Exit_Time", "Re_Entry", "Status"])
    df.to_csv(attendance_file, index=False)

mylist = os.listdir(path)
for cls in mylist:
    currentImg = cv2.imread(f"{path}/{cls}")
    images.append(currentImg)
    ClassNames.append(os.path.splitext(cls)[0])
# print(ClassNames)
# cv2.imshow("image",images[2])
# cv2.waitKey(0)

def encoder(images):
    encode_list = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encode_list.append(encode)
    return encode_list

def mark_attendance(name):
    now = datetime.now()
    date_today = now.strftime('%Y-%m-%d')
    entry_time = now.strftime('%H:%M:%S')
    
    # Read existing attendance data
    df = pd.read_csv(attendance_file)
    
    # Check if the name already exists in today's attendance
    if (df['Name'] == name).any() and (df['Date'] == date_today).any():
        # Update the Exit_Time and Status if the name already exists
        df.loc[(df['Name'] == name) & (df['Date'] == date_today), 'Exit_Time'] = entry_time
        df.loc[(df['Name'] == name) & (df['Date'] == date_today), 'Status'] = 'OUT'
        df.loc[(df['Name'] == name) & (df['Date'] == date_today), 'Re_Entry'] = True
    else:
        # Append new entry for the attendance
        new_entry = {
            "Date": date_today,
            "Name": name,
            "Entry_Time": entry_time,
            "Exit_Time": "",
            "Re_Entry": False,
            "Status": "IN"
        }
        df = df.append(new_entry, ignore_index=True)
    
    # Save the updated DataFrame back to CSV
    df.to_csv(attendance_file, index=False)

# Encode images
encodeList = encoder(images)
# print(len(encodeList))
print("Encoding Complete")

# Start video capture
cap = cv2.VideoCapture(0)

while True:
    success, img = cap.read()
    imgS = cv2.resize(img, (0, 0), None, 0.25, 0.25)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)
    
    face_Current_Frame = face_recognition.face_locations(imgS)
    encoding_Current_Frame = face_recognition.face_encodings(imgS, face_Current_Frame)
    
    for encode_face, face_loc in zip(encoding_Current_Frame, face_Current_Frame):
        matches = face_recognition.compare_faces(encodeList, encode_face)
        face_Distance = face_recognition.face_distance(encodeList, encode_face)
        match_index = np.argmin(face_Distance)
        
        if matches[match_index]:
            name = ClassNames[match_index].upper()
            mark_attendance(name)
            print(f"Attendance Marked for {name}")
            y1, x2, y2, x1 = face_loc
            y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)
            cv2.putText(img, name, (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 0.8, (255, 255, 255), 2)
    
    cv2.imshow("video", img)
    if cv2.waitKey(1) & 0xff == ord('q'):
        break


Encoding Complete


AttributeError: 'DataFrame' object has no attribute 'append'

In [2]:
import numpy as np
import cv2
import face_recognition
import os
import pandas as pd
from datetime import datetime, timedelta
import time  # Import time module for sleep

path = "data"
images = []
ClassNames = []

# Check if the attendance.csv file exists; if not, create it with the required columns
attendance_file = "attendance.csv"
if not os.path.exists(attendance_file):
    # Create a new DataFrame with the required columns
    df = pd.DataFrame(columns=["Date", "Name", "Entry_Time", "Exit_Time", "Re_Entry", "Status"])
    df.to_csv(attendance_file, index=False)

mylist = os.listdir(path)
for cls in mylist:
    currentImg = cv2.imread(f"{path}/{cls}")
    images.append(currentImg)
    ClassNames.append(os.path.splitext(cls)[0])

def encoder(images):
    encode_list = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encode_list.append(encode)
    return encode_list

def mark_attendance(name):
    now = datetime.now()
    date_today = now.strftime('%Y-%m-%d')
    entry_time = now.strftime('%H:%M:%S')
    
    # Read existing attendance data
    df = pd.read_csv(attendance_file)
    
    # Ensure the Exit_Time column is treated as string (object type)
    if 'Exit_Time' not in df.columns:
        df['Exit_Time'] = pd.Series(dtype='object')
    
    # Check if the name already exists in today's attendance
    if (df['Name'] == name).any() and (df['Date'] == date_today).any():
        # Get the current entry for this name
        entry = df[(df['Name'] == name) & (df['Date'] == date_today)].iloc[0]
        
        if entry['Status'] == "IN":
            # Check if Exit_Time is present
            if pd.isnull(entry['Exit_Time']):
                # If Exit_Time is not present, wait for 10 minutes before allowing to mark as OUT
                entry_time_dt = datetime.strptime(entry_time, '%H:%M:%S')
                time_difference = now - entry_time_dt

                if time_difference >= timedelta(minutes=10):
                    # It's been 10 minutes, mark as OUT
                    df.loc[(df['Name'] == name) & (df['Date'] == date_today), 'Exit_Time'] = str(entry_time)  # Cast to string
                    df.loc[(df['Name'] == name) & (df['Date'] == date_today), 'Status'] = "OUT"
                    print(f"Updated Exit_Time for {name} to {entry_time} and status to OUT.")
                else:
                    print(f"{name} cannot exit yet; please wait for 10 minutes.")
            else:
                # If Exit_Time is present, do nothing (or handle as needed)
                print(f"{name} is already marked OUT.")
        
        elif entry['Status'] == "OUT":
            # Mark ReEntry as True and change status to IN
            df.loc[(df['Name'] == name) & (df['Date'] == date_today), 'Re_Entry'] = True
            df.loc[(df['Name'] == name) & (df['Date'] == date_today), 'Status'] = "IN"
            print(f"Marked {name} as IN again (ReEntry).")
        
    else:
        # Create a new entry for attendance
        new_entry = pd.DataFrame({
            "Date": [date_today],
            "Name": [name],
            "Entry_Time": [entry_time],
            "Exit_Time": [None],
            "Re_Entry": [False],
            "Status": ["IN"]
        })
        
        # Concatenate the new entry to the existing DataFrame
        df = pd.concat([df, new_entry], ignore_index=True)
        print(f"Added new entry for {name}.")

    # Save the updated DataFrame back to CSV
    df.to_csv(attendance_file, index=False)

# Encode images
encodeList = encoder(images)
print("Encoding Complete")

# Start video capture
cap = cv2.VideoCapture(0)

while True:
    success, img = cap.read()
    imgS = cv2.resize(img, (0, 0), None, 0.25, 0.25)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)
    
    face_Current_Frame = face_recognition.face_locations(imgS)
    encoding_Current_Frame = face_recognition.face_encodings(imgS, face_Current_Frame)
    
    for encode_face, face_loc in zip(encoding_Current_Frame, face_Current_Frame):
        matches = face_recognition.compare_faces(encodeList, encode_face)
        face_Distance = face_recognition.face_distance(encodeList, encode_face)
        match_index = np.argmin(face_Distance)
        
        if matches[match_index]:
            name = ClassNames[match_index].upper()
            mark_attendance(name)
            print(f"Attendance Marked for {name}")
            time.sleep(10)  # Add a 10-second delay after recognizing a face
            y1, x2, y2, x1 = face_loc
            y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)
            cv2.putText(img, name, (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 0.8, (255, 255, 255), 2)
    
    cv2.imshow("video", img)
    if cv2.waitKey(1) & 0xff == ord('q'):
        break


Encoding Complete
Added new entry for NADEEM.
Attendance Marked for NADEEM


C:\Users\nadee\AppData\Local\Temp\ipykernel_700\1933501034.py:60: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '12:31:26' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[(df['Name'] == name) & (df['Date'] == date_today), 'Exit_Time'] = str(entry_time)  # Cast to string


Updated Exit_Time for NADEEM to 12:31:26 and status to OUT.
Attendance Marked for NADEEM
Marked NADEEM as IN again (ReEntry).
Attendance Marked for NADEEM
NADEEM is already marked OUT.
Attendance Marked for NADEEM
NADEEM is already marked OUT.
Attendance Marked for NADEEM
NADEEM is already marked OUT.
Attendance Marked for NADEEM


In [ ]:
import numpy as np
import cv2
import face_recognition
import os
import pandas as pd
from datetime import datetime, timedelta
import time  # Import the time module

path = "data"
images = []
ClassNames = []

# Check if the attendance.csv file exists; if not, create it with the required columns
attendance_file = "attendance.csv"
if not os.path.exists(attendance_file):
    # Create a new DataFrame with the required columns
    df = pd.DataFrame(columns=["Date", "Name", "Entry_Time", "Exit_Time", "Re_Entry", "Status", "Last_Entry_Time"])
    df.to_csv(attendance_file, index=False)

mylist = os.listdir(path)
for cls in mylist:
    currentImg = cv2.imread(f"{path}/{cls}")
    images.append(currentImg)
    ClassNames.append(os.path.splitext(cls)[0])

def encoder(images):
    encode_list = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encode_list.append(encode)
    return encode_list

def mark_attendance(name):
    now = datetime.now()
    date_today = now.strftime('%Y-%m-%d')
    entry_time = now.strftime('%H:%M:%S')
    
    # Read existing attendance data
    df = pd.read_csv(attendance_file)
    
    # Ensure the Exit_Time column is treated as string (object type)
    if 'Exit_Time' not in df.columns:
        df['Exit_Time'] = pd.Series(dtype='object')
    
    # Check if the name already exists in today's attendance
    if (df['Name'] == name).any() and (df['Date'] == date_today).any():
        # Get the current entry for this name
        entry = df[(df['Name'] == name) & (df['Date'] == date_today)].iloc[0]
        
        if entry['Status'] == "IN":
            # Check if Exit_Time is present
            if pd.isnull(entry['Exit_Time']):
                # If Exit_Time is not present, check the time since entry
                last_entry_time = datetime.strptime(entry['Last_Entry_Time'], '%H:%M:%S')
                if now - last_entry_time >= timedelta(minutes=10):
                    # If 10 minutes have passed, allow updating Exit_Time
                    df.loc[(df['Name'] == name) & (df['Date'] == date_today), 'Exit_Time'] = str(entry_time)  # Cast to string
                    df.loc[(df['Name'] == name) & (df['Date'] == date_today), 'Status'] = "OUT"
                    print(f"Updated Exit_Time for {name} to {entry_time} and status to OUT.")
                else:
                    print(f"{name} cannot exit yet; please wait 10 minutes since entry.")
            else:
                # If Exit_Time is present, do nothing (or handle as needed)
                print(f"{name} is already marked OUT.")
        
        elif entry['Status'] == "OUT":
            # Mark ReEntry as True and change status to IN
            df.loc[(df['Name'] == name) & (df['Date'] == date_today), 'Re_Entry'] = True
            df.loc[(df['Name'] == name) & (df['Date'] == date_today), 'Status'] = "IN"
            df.loc[(df['Name'] == name) & (df['Date'] == date_today), 'Last_Entry_Time'] = entry_time  # Update Last Entry Time
            print(f"Marked {name} as IN again (ReEntry).")
        
    else:
        # Create a new entry for attendance
        new_entry = pd.DataFrame({
            "Date": [date_today],
            "Name": [name],
            "Entry_Time": [entry_time],
            "Exit_Time": [None],
            "Re_Entry": [False],
            "Status": ["IN"],
            "Last_Entry_Time": [entry_time]  # Set the Last Entry Time
        })
        
        # Concatenate the new entry to the existing DataFrame
        df = pd.concat([df, new_entry], ignore_index=True)
        print(f"Added new entry for {name}.")

    # Save the updated DataFrame back to CSV
    df.to_csv(attendance_file, index=False)

# Encode images
encodeList = encoder(images)
print("Encoding Complete")

# Start video capture
cap = cv2.VideoCapture(0)

while True:
    success, img = cap.read()
    imgS = cv2.resize(img, (0, 0), None, 0.25, 0.25)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)
    
    face_Current_Frame = face_recognition.face_locations(imgS)
    encoding_Current_Frame = face_recognition.face_encodings(imgS, face_Current_Frame)
    
    for encode_face, face_loc in zip(encoding_Current_Frame, face_Current_Frame):
        matches = face_recognition.compare_faces(encodeList, encode_face)
        face_Distance = face_recognition.face_distance(encodeList, encode_face)
        match_index = np.argmin(face_Distance)
        
        if matches[match_index]:
            name = ClassNames[match_index].upper()
            print(f"Face recognized: {name}. Waiting for 10 seconds before marking attendance...")
            time.sleep(10)  # Add a delay of 10 seconds
            
            mark_attendance(name)
            print(f"Attendance Marked for {name}")
            y1, x2, y2, x1 = face_loc
            y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)
            cv2.putText(img, name, (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 0.8, (255, 255, 255), 2)
    
    cv2.imshow("video", img)
    if cv2.waitKey(1) & 0xff == ord('q'):
        break


### Without delay


In [1]:
import numpy as np
import cv2
import face_recognition
import os
import pandas as pd
from datetime import datetime

path = "data"
images = []
ClassNames = []

# Check if the attendance.csv file exists; if not, create it with the required columns
attendance_file = "attendance.csv"
if not os.path.exists(attendance_file):
    # Create a new DataFrame with the required columns
    df = pd.DataFrame(columns=["Date", "Name", "Entry_Time", "Exit_Time", "Re_Entry", "Status"])
    df.to_csv(attendance_file, index=False)

mylist = os.listdir(path)
for cls in mylist:
    currentImg = cv2.imread(f"{path}/{cls}")
    images.append(currentImg)
    ClassNames.append(os.path.splitext(cls)[0])

def encoder(images):
    encode_list = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encode_list.append(encode)
    return encode_list

def mark_attendance(name):
    now = datetime.now()
    date_today = now.strftime('%Y-%m-%d')
    entry_time = now.strftime('%H:%M:%S')
    
    # Read existing attendance data
    df = pd.read_csv(attendance_file)
    
    # Check if the name already exists in today's attendance
    if (df['Name'] == name).any() and (df['Date'] == date_today).any():
        # Get the current entry for this name
        entry = df[(df['Name'] == name) & (df['Date'] == date_today)].iloc[0]
        
        if entry['Status'] == "IN":
            # Check if Exit_Time is present
            if pd.isnull(entry['Exit_Time']):
                # If Exit_Time is not present, update it and change status to OUT
                df.loc[(df['Name'] == name) & (df['Date'] == date_today), 'Exit_Time'] = entry_time
                df.loc[(df['Name'] == name) & (df['Date'] == date_today), 'Status'] = "OUT"
                print(f"Updated Exit_Time for {name} to {entry_time} and status to OUT.")
            else:
                # If Exit_Time is present, do nothing (or handle as needed)
                print(f"{name} is already marked OUT.")
        
        elif entry['Status'] == "OUT":
            # Mark ReEntry as True and change status to IN
            df.loc[(df['Name'] == name) & (df['Date'] == date_today), 'Re_Entry'] = True
            df.loc[(df['Name'] == name) & (df['Date'] == date_today), 'Status'] = "IN"
            print(f"Marked {name} as IN again (ReEntry).")
        
    else:
        # Create a new entry for attendance
        new_entry = pd.DataFrame({
            "Date": [date_today],
            "Name": [name],
            "Entry_Time": [entry_time],
            "Exit_Time": [None],
            "Re_Entry": [False],
            "Status": ["IN"]
        })
        
        # Concatenate the new entry to the existing DataFrame
        df = pd.concat([df, new_entry], ignore_index=True)
        print(f"Added new entry for {name}.")

    # Save the updated DataFrame back to CSV
    df.to_csv(attendance_file, index=False)

# Encode images
encodeList = encoder(images)
print("Encoding Complete")

# Start video capture
cap = cv2.VideoCapture(0)

while True:
    success, img = cap.read()
    imgS = cv2.resize(img, (0, 0), None, 0.25, 0.25)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)
    
    face_Current_Frame = face_recognition.face_locations(imgS)
    encoding_Current_Frame = face_recognition.face_encodings(imgS, face_Current_Frame)
    
    for encode_face, face_loc in zip(encoding_Current_Frame, face_Current_Frame):
        matches = face_recognition.compare_faces(encodeList, encode_face)
        face_Distance = face_recognition.face_distance(encodeList, encode_face)
        match_index = np.argmin(face_Distance)
        
        if matches[match_index]:
            name = ClassNames[match_index].upper()
            mark_attendance(name)
            print(f"Attendance Marked for {name}")
            y1, x2, y2, x1 = face_loc
            y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)
            cv2.putText(img, name, (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 0.8, (255, 255, 255), 2)
    
    cv2.imshow("video", img)
    if cv2.waitKey(1) & 0xff == ord('q'):
        break


Encoding Complete
NADEEM is already marked OUT.
Attendance Marked for NADEEM
NADEEM is already marked OUT.
Attendance Marked for NADEEM
NADEEM is already marked OUT.
Attendance Marked for NADEEM
NADEEM is already marked OUT.
Attendance Marked for NADEEM
NADEEM is already marked OUT.
Attendance Marked for NADEEM
NADEEM is already marked OUT.
Attendance Marked for NADEEM
NADEEM is already marked OUT.
Attendance Marked for NADEEM
NADEEM is already marked OUT.
Attendance Marked for NADEEM
NADEEM is already marked OUT.
Attendance Marked for NADEEM
NADEEM is already marked OUT.
Attendance Marked for NADEEM
NADEEM is already marked OUT.
Attendance Marked for NADEEM
NADEEM is already marked OUT.
Attendance Marked for NADEEM
NADEEM is already marked OUT.
Attendance Marked for NADEEM
NADEEM is already marked OUT.
Attendance Marked for NADEEM
NADEEM is already marked OUT.
Attendance Marked for NADEEM
NADEEM is already marked OUT.
Attendance Marked for NADEEM
NADEEM is already marked OUT.
Attendan

In [6]:
import numpy as np
import cv2
import face_recognition
import os
import pandas as pd
from datetime import datetime

path = "data"
images = []
ClassNames = []

# Check if the attendance.csv file exists; if not, create it with the required columns
attendance_file = "attendance.csv"
if not os.path.exists(attendance_file):
    # Create a new DataFrame with the required columns
    df = pd.DataFrame(columns=["Date", "Name", "Entry_Time", "Exit_Time", "Re_Entry", "Status"])
    df.to_csv(attendance_file, index=False)

mylist = os.listdir(path)
for cls in mylist:
    currentImg = cv2.imread(f"{path}/{cls}")
    images.append(currentImg)
    ClassNames.append(os.path.splitext(cls)[0])

def encoder(images):
    encode_list = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encode_list.append(encode)
    return encode_list

def mark_attendance(name):
    now = datetime.now()
    date_today = now.strftime('%Y-%m-%d')
    entry_time = now.strftime('%H:%M:%S')
    
    # Read existing attendance data
    df = pd.read_csv(attendance_file)
    
    # checking if name and date is already present
    if (df['Name'] == name).any() and (df['Date'] == date_today).any():
        # getting this persons data from table
        entry = df[(df['Name'] == name) & (df['Date'] == date_today)].iloc[0]
        
        if entry['Status'] == "IN":
            # Check if Exit_Time is present
            if pd.isnull(entry['Exit_Time']):
                # If Exit_Time is not present, update it and change status to OUT
                df.loc[(df['Name'] == name) & (df['Date'] == date_today), 'Exit_Time'] = now.strftime('%H:%M:%S')
                df.loc[(df['Name'] == name) & (df['Date'] == date_today), 'Status'] = "OUT"
                print(f"Updated Exit_Time for {name} to {entry_time} and status to OUT.")
            else:
                # If Exit_Time is present, do nothing (or handle as needed)
                print(f"{name} has taken re-exit.")
                df.loc[(df['Name'] == name) & (df['Date'] == date_today), 'Exit_Time'] = now.strftime('%H:%M:%S')
                df.loc[(df['Name'] == name) & (df['Date'] == date_today), 'Status'] = "OUT"
        
        elif entry['Status'] == "OUT":
            # Mark ReEntry as True and change status to IN
            df.loc[(df['Name'] == name) & (df['Date'] == date_today), 'Re_Entry'] = True
            df.loc[(df['Name'] == name) & (df['Date'] == date_today), 'Status'] = "IN"
            print(f"Marked {name} as IN again (ReEntry).")
        
    else:
        # Create a new entry for attendance
        new_entry = pd.DataFrame({
            "Date": [date_today],
            "Name": [name],
            "Entry_Time": [entry_time],
            "Exit_Time": [None],
            "Re_Entry": [False],
            "Status": ["IN"]
        })
        
        # Concatenate the new entry to the existing DataFrame
        df = pd.concat([df, new_entry], ignore_index=True)
        print(f"Added new entry for {name}.")

    # Save the updated DataFrame back to CSV
    df.to_csv(attendance_file, index=False)

# Encode images
encodeList = encoder(images)
print("Encoding Complete")

# Start video capture
cap = cv2.VideoCapture(0)

while True:
    success, img = cap.read()
    imgS = cv2.resize(img, (0, 0), None, 0.25, 0.25)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)
    
    face_Current_Frame = face_recognition.face_locations(imgS)
    encoding_Current_Frame = face_recognition.face_encodings(imgS, face_Current_Frame)
    
    for encode_face, face_loc in zip(encoding_Current_Frame, face_Current_Frame):
        matches = face_recognition.compare_faces(encodeList, encode_face)
        face_Distance = face_recognition.face_distance(encodeList, encode_face)
        match_index = np.argmin(face_Distance)
        
        if matches[match_index]:
            name = ClassNames[match_index].upper()
            mark_attendance(name)
            print(f"Attendance Marked for {name}")
            y1, x2, y2, x1 = face_loc
            y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)
            cv2.putText(img, name, (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 0.8, (255, 255, 255), 2)
    
    cv2.imshow("video", img)
    if cv2.waitKey(1) & 0xff == ord('q'):
        break


Encoding Complete
NADEEM has taken re-exit.
Attendance Marked for NADEEM
Marked NADEEM as IN again (ReEntry).
Attendance Marked for NADEEM
NADEEM has taken re-exit.
Attendance Marked for NADEEM
Marked NADEEM as IN again (ReEntry).
Attendance Marked for NADEEM
NADEEM has taken re-exit.
Attendance Marked for NADEEM


In [ ]:
import numpy as np
import cv2
import face_recognition
import os
import pandas as pd
from datetime import datetime, timedelta
import time  # Import for delay

path = "data"
images = []
ClassNames = []

# Check if the attendance.csv file exists; if not, create it with the required columns
attendance_file = "attendance.csv"
if not os.path.exists(attendance_file):
    df = pd.DataFrame(columns=["Date", "Name", "Entry_Time", "Exit_Time", "Re_Entry", "Status"])
    df.to_csv(attendance_file, index=False)

mylist = os.listdir(path)
for cls in mylist:
    currentImg = cv2.imread(f"{path}/{cls}")
    images.append(currentImg)
    ClassNames.append(os.path.splitext(cls)[0])

def encoder(images):
    encode_list = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encode_list.append(encode)
    return encode_list

def mark_attendance(name):
    now = datetime.now()
    date_today = now.strftime('%Y-%m-%d')
    entry_time = now.strftime('%H:%M:%S')
    
    # Read existing attendance data
    df = pd.read_csv(attendance_file)
    
    # Check if an entry already exists for this person today
    if (df['Name'] == name).any() and (df['Date'] == date_today).any():
        # Get this person's entry from the table
        entry = df[(df['Name'] == name) & (df['Date'] == date_today)].iloc[0]
        
        entry_time_dt = datetime.strptime(entry['Entry_Time'], '%H:%M:%S') if pd.notna(entry['Entry_Time']) else None
        exit_time_dt = datetime.strptime(entry['Exit_Time'], '%H:%M:%S') if pd.notna(entry['Exit_Time']) else None
        
        # If the person is marked as "IN" (present in the system)
        if entry['Status'] == "IN":
            if pd.isnull(entry['Exit_Time']):
                # Ensure 10 minutes have passed since entry
                if entry_time_dt and (now - entry_time_dt) >= timedelta(minutes=1):
                    df.loc[(df['Name'] == name) & (df['Date'] == date_today), 'Exit_Time'] = entry_time
                    df.loc[(df['Name'] == name) & (df['Date'] == date_today), 'Status'] = "OUT"
                    print(f"Updated Exit_Time for {name} to {entry_time} and status to OUT.")
                else:
                    time_diff = now - entry_time_dt if entry_time_dt else timedelta(0)
                    print(f"Cannot update Exit_Time for {name}. Only {time_diff.seconds // 60} minutes have passed since Entry.")
            else:
                # If exit time exists and they are trying to re-exit (Re-exit scenario)
                print(f"{name} has already exited earlier today.")
        
        elif entry['Status'] == "OUT":
            # Ensure 10 minutes have passed since exit before re-entry
            if exit_time_dt and (now - exit_time_dt) >= timedelta(minutes=1):
                df.loc[(df['Name'] == name) & (df['Date'] == date_today), 'Re_Entry'] = True
                df.loc[(df['Name'] == name) & (df['Date'] == date_today), 'Status'] = "IN"
                print(f"Marked {name} as IN again (Re-Entry).")
            else:
                time_diff = now - exit_time_dt if exit_time_dt else timedelta(0)
                print(f"Cannot mark Re-Entry for {name}. Only {time_diff.seconds // 60} minutes have passed since Exit.")
    else:
        # If no record exists for this person today, create a new one
        new_entry = pd.DataFrame({
            "Date": [date_today],
            "Name": [name],
            "Entry_Time": [entry_time],
            "Exit_Time": [None],
            "Re_Entry": [False],
            "Status": ["IN"]
        })
        
        # Concatenate the new entry to the existing DataFrame
        df = pd.concat([df, new_entry], ignore_index=True)
        print(f"Added new entry for {name}.")
    
    # Save the updated DataFrame back to CSV
    df.to_csv(attendance_file, index=False)


# Encode images
encodeList = encoder(images)
print("Encoding Complete")

# Start video capture
cap = cv2.VideoCapture(0)

while True:
    success, img = cap.read()
    imgS = cv2.resize(img, (0, 0), None, 0.25, 0.25)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)
    
    face_Current_Frame = face_recognition.face_locations(imgS)
    encoding_Current_Frame = face_recognition.face_encodings(imgS, face_Current_Frame)
    
    for encode_face, face_loc in zip(encoding_Current_Frame, face_Current_Frame):
        matches = face_recognition.compare_faces(encodeList, encode_face)
        face_Distance = face_recognition.face_distance(encodeList, encode_face)
        match_index = np.argmin(face_Distance)
        
        if matches[match_index]:
            name = ClassNames[match_index].upper()
            mark_attendance(name)
            print(f"Attendance Marked for {name}")
            
            # Add 10-second delay after recognizing a face
            time.sleep(10)
            
            y1, x2, y2, x1 = face_loc
            y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)
            cv2.putText(img, name, (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 0.8, (255, 255, 255), 2)
    
    cv2.imshow("video", img)
    if cv2.waitKey(1) & 0xff == ord('q'):
        break


Encoding Complete
NADEEM has already exited earlier today.
Attendance Marked for NADEEM
NADEEM has already exited earlier today.
Attendance Marked for NADEEM
NADEEM has already exited earlier today.
Attendance Marked for NADEEM
NADEEM has already exited earlier today.
Attendance Marked for NADEEM
NADEEM has already exited earlier today.
Attendance Marked for NADEEM
NADEEM has already exited earlier today.
Attendance Marked for NADEEM
NADEEM has already exited earlier today.
Attendance Marked for NADEEM
NADEEM has already exited earlier today.
Attendance Marked for NADEEM
NADEEM has already exited earlier today.
Attendance Marked for NADEEM
NADEEM has already exited earlier today.
Attendance Marked for NADEEM
NADEEM has already exited earlier today.
Attendance Marked for NADEEM
